In [1]:
image_path = "sample.jpg"
import torch

In [2]:
import detectron2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import cv2
import numpy as np

# Setup configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0001  # Lower threshold
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 2000  # Increase number of proposals post-NMS
cfg.MODEL.RPN.PRE_NMS_TOPK_TEST = 2000   # Increase number of proposals pre-NMS
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024  # Increase batch size per image
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Create predictor
predictor = DefaultPredictor(cfg)

# Load image
image = cv2.imread(image_path)
lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
cl = clahe.apply(l)
limg = cv2.merge((cl, a, b))
image_contrast = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

# Resize image
image = cv2.resize(image_contrast, (800, 800))

# Perform prediction
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply histogram equalization
#equalized = cv2.equalizeHist(gray)

# Convert back to BGR
#image_preprocessed = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
#image = image_preprocessed
#image = cv2.resize(image, (800, 800))

outputs = predictor(image)

# Visualize predictions
instances = outputs["instances"].to("cpu")
boxes = instances.pred_boxes.tensor.numpy()
scores = instances.scores.numpy()
classes = instances.pred_classes.numpy()

indices = cv2.dnn.NMSBoxes(boxes.tolist(), scores.tolist(), score_threshold=0.00001, nms_threshold=0.7)

# Draw bounding boxes
for i in indices.flatten():
    x1, y1, x2, y2 = boxes[i]
    if scores[i] < 0.4:  # Adjusted threshold
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
        cv2.putText(image, f"Score: {scores[i]:.2f}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

# Save and display the image
output_image_path = "/mnt/data/detected_logos_detectron2.png"
cv2.imwrite(output_image_path, image)
cv2.imshow("Detected Logos", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


/opt/miniconda3/envs/venv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
